In [1]:
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import tensorflow.contrib.eager as tfe

%matplotlib inline
plt.style.use("fivethirtyeight")
sns.set_context("notebook")

C:\Users\e739218\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
tfe.enable_eager_execution()

In [3]:
data = {}
with open("../examples/data/birth_life_2010.txt", "r") as file:
    for line in file.readlines()[1:]:
        temp = line.split("\t")
        data[temp[0]] = (float(temp[1]), float(temp[2][:-1]))
        
data = pd.DataFrame(data).transpose()
data.columns = ["birth_rate", "life_expectancy"]
data.head()

,birth_rate,life_expectancy
Afghanistan,6.288,48.282195
Albania,1.536,76.900951
Algeria,2.264,72.852537
Angola,5.443,50.653659
Argentina,2.211,75.632146


In [4]:
# Convert dataframe into features and targets nd-arrays
X_input = data["birth_rate"].values
target = data["life_expectancy"].values

# Shuffle the data
np.random.seed(123)
perms = np.random.permutation(X_input.shape[0])
X_input = X_input[perms]
target = target[perms]

X_input.shape, target.shape

((190,), (190,))

In [5]:
def prediction(x):
    return w * x + b

# Define loss functions
def huber_loss(y, y_pred, delta=1.0):
    residuals = tf.abs(y - y_pred)
    if tf.square(residuals) <= delta:
        return 0.5 * tf.square(residuals)
    return delta * (residuals - 0.5 * delta)


def squared_loss(y, y_pred):
    return (y - y_pred) ** 2

# Define Dataset object
dataset = tf.data.Dataset.from_tensor_slices((X_input, target))

# Define parameters
w = tfe.Variable(0, dtype=tf.float64, name="weight")
b = tfe.Variable(0, dtype=tf.float64, name="bias")

# Build model
def train(loss_fn):
    # Define optimizer
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
    
    # Define loss for each example
    def loss_for_example(x, y):
        y_pred = prediction(x)
        return loss_fn(y, y_pred)
    
    # Define gradient fn to be used when applying optimizer
    grad_fn = tfe.implicit_value_and_gradients(loss_for_example)
    
    costs = []
    for epoch in range(200):
        cost_epoch = 0
        iterator = tfe.Iterator(dataset)
        for x_i, y_i in iterator:
            loss, gradients = grad_fn(x_i, y_i)
            optimizer.apply_gradients(gradients)
            cost_epoch += loss
        
        costs.append(cost_epoch)
    
    return costs

cost = train(squared_loss)

In [6]:
w, b

(<tf.Variable 'weight:0' shape=() dtype=float64, numpy=-5.363653241996434>,
 <tf.Variable 'bias:0' shape=() dtype=float64, numpy=86.06626790267111>)